In [2]:
from PIL import Image
from pathlib import Path
import os

In [3]:
### TODO entender mejor el uso de Path y Pillow
#   -- revisar documentación https://pillow.readthedocs.io/en/stable/
#      modulo Image
#   -- esto se convertirá en un .py eventualmente

def optimizar_squoosh_local(carpeta_entrada, calidad_inicial=95, calidad_minima=80, tamano_max_kb=500):
    carpeta_entrada = Path(carpeta_entrada)
    carpeta_salida = carpeta_entrada / "imgs_optimizadas"
    carpeta_salida.mkdir(parents=True, exist_ok=True)

    for ruta in carpeta_entrada.glob("*.*"):
        if ruta.suffix.lower() not in (".jpg", ".jpeg", ".png"):
            continue

        with Image.open(ruta) as img:
            # --- Convertir PNG a JPG si aplica ---
            if ruta.suffix.lower() == ".png":
                
                # Si tiene canal alpha o es P, pasar a RBG primero 
                if img.mode in ("RGBA", "P"):
                    img = img.convert("RGB")
                nueva_ruta = ruta.with_suffix(".jpg")
            else:
                nueva_ruta = ruta

            # --- Redimensionar ---
                # Si la imagen es más ancha que alta, el ancho será 1500 px, sino, será 1000 px
            w, h = img.size

            ancho_objetivo = 1500 if w >= h else 1000
            if w > ancho_objetivo:
                scale = ancho_objetivo / w
                nuevo_w = ancho_objetivo
                nuevo_h = int(h * scale)
                img = img.resize((nuevo_w, nuevo_h), Image.LANCZOS)
            else:
                nuevo_w, nuevo_h = w, h

            # --- Normalizar modo de color para JPG ---
            if img.mode in ("RGBA", "P"):
                img = img.convert("RGB")

            salida = carpeta_salida / nueva_ruta.name

            # --- Ajuste de calidad dinámico ---
            calidad = calidad_inicial
            while calidad >= calidad_minima:
                img.save(salida, "JPEG", optimize=True, progressive=True, quality=calidad)
                tamano_kb = os.path.getsize(salida) / 1024
                if tamano_kb <= tamano_max_kb:
                    break
                calidad -= 3

            # --- Alerta si no se logró bajar de 500 KB ---
            if tamano_kb > tamano_max_kb:
                print(f"⚠️  {ruta.name}: No pudo bajar de {tamano_max_kb} KB "
                      f"(quedó en {tamano_kb:.0f} KB, calidad {calidad_minima})")

            print(f"✅ {nueva_ruta.name}: {nuevo_w}x{nuevo_h}px - {tamano_kb:.0f} KB")


optimizar_squoosh_local(r"C:\Users\juant\Downloads\carruseles_infografias")


✅ 1.jpg: 1000x1250px - 297 KB
✅ 2.jpg: 1000x1250px - 285 KB
✅ 3.jpg: 1000x1250px - 331 KB
✅ 4.jpg: 1000x1250px - 242 KB
✅ 5.jpg: 1000x1250px - 167 KB
✅ 6.jpg: 1000x1250px - 425 KB
✅ 7.jpg: 1000x1250px - 347 KB
